<a href="https://colab.research.google.com/github/Rripped/PromptRefining/blob/main/PromptRefining_Stable.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

TODO: system message hinzufügen (whatsapp); in sys mess: originaler prompt, img mehr wie prompt, neuer prompt präziser

--> assistant, (moderation?)

-----------------------------------------------------------------------------


This notebook sends a prompt to ChatGPT 3.5 and creates images from scratch given a prompt as well as variations of given images. (See print statements at the end of the notebook)

If command fails, go to Runtime - Change Runtime type - GPU T4

In [ ]:
!nvidia-smi

necessary installs

In [ ]:
!pip install diffusers==0.11.1
!pip install transformers scipy ftfy accelerate
!pip install openai cohere tiktoken jq requests

necessary imports

In [ ]:
import json
import os
import openai
from openai import OpenAI
import requests
import shutil
import torch
from diffusers import StableDiffusionPipeline

In [ ]:
client = OpenAI(api_key=os.environ["OAI"])

In [ ]:
pipe = StableDiffusionPipeline.from_pretrained("stabilityai/stable-diffusion-2-1", torch_dtype=torch.float16)
pipe = pipe.to("cuda")

In [ ]:
def getImageDallE(newPrompt):
  return client.images.generate(model="dall-e-2", prompt=newPrompt, size="256x256", quality="standard", n=1)

In [ ]:
initialPrompt = "a mouse eating cheese"
filename = "output.png"

In [ ]:
def getImageStableDiff(prompt, filename):
  image = pipe(prompt).images[0]
  image.save(filename)
  return image

image = getImageStableDiff(initialPrompt, filename)
image

In [ ]:
def getNewPrompt(imageUrl):
  print("initial prompt: " + initialPrompt)
  return client.chat.completions.create(
  model="gpt-4-vision-preview",
  messages=[
      {
      "role": "system",
      "content": "You will have to compare an AI generated image with an original prompt. The prompt is marked by the preceding tag [PROMPT]. The goal is to filter out all differences of a prompt to the image. Tagged by [DIFFERENCES] you will return a list of bulletpoints, with short differences between the prompt and the image. Be short and if no differences exist, leave the differences empty. In a second paragraph marked with [NEW PROMPT], design a new prompt, which precisely filters out the differences, so that the new prompt can be used to create a new image, which is closer to the original prompt. In the end there should be no more things in the picture than asked for in the original prompt. Provide no more information than defined here."

      },
    {
      "role": "user",
      "content": [
        {"type": "text", "text": str("[PROMPT] " + initialPrompt)},
        {
          "type": "image_url",
          "image_url": {
            "url": imageUrl,
          },
        },
      ]}
  ],
  max_tokens=150,
  temperature=0.2,
  top_p=0.3
)

In [ ]:
def download(url, file_name):
    res = requests.get(url, stream = True)

    if res.status_code == 200:
      with open(file_name,'wb') as f:
        shutil.copyfileobj(res.raw, f)
      print('Image sucessfully downloaded: ', file_name)
    else:
      print('Image couldn\'t be retrieved')

You will have to compare an AI generated image with an original prompt. The prompt is marked by the preceding tag [PROMPT].

The goal is to filter out all differences of a prompt to the image. Tagged by [DIFFERENCES] you will return a list of bulletpoints, with short differences between the prompt and the image. Be short and if no differences exist, leave the differences empty.

In a second paragraph marked with [NEW PROMPT], design a new prompt, which precisely filters out the differences, so that the new prompt can be used to create a new image, which is closer to the original prompt. In the end there should be no more things in the picture than asked for in the original prompt.

Provide no more information than defined here. Please answer with "Yes" if you understood.


You are given a prompt and an image. Your task is to improve the wording of the prompt so that Dall-E can generate a more realistic image of the given prompt. Respond with the newly generated prompt only.

In [ ]:
def loop(promptImg):
  i = 0
  #image = getImageDallE(initialPrompt).data[0].url
  image = getImageStableDiff(initialPrompt, filename)

  while i < 5:
    print(f"iteration " + str(i) + ": \n Image: ")
    image
    responseChat = getNewPrompt(filename)
    newPrompt = responseChat.choices[0].message.content
    print(f"\nnew prompt: " + newPrompt)
    #image = getImageDallE(newPrompt).data[0].url
    image = getImageStableDiff(newPrompt, filename)
    i += 1

loop(initialPrompt)